### Install tendb
Must install latest tendb before running the commands
Install from released package from pypi
```bash
$ pip install -i https://test.pypi.org/simple/ tendb
```
Read local setup.py and install tendb
```bash
pip install /mnt/c/Users/hkver/Documents/dbai/dbaistuff/py/tendb
```
Install from local wheel file
```bash
pip install /mnt/c/Users/hkver/Documents/dbai/dbaistuff/py/tendb/dist/tendb-0.0.1-py3-none-any.whl
```

Read Arrow table

In [ ]:
import pyarrow as pa
from pyarrow import csv

In [ ]:
pa.show_versions()

In [ ]:
csv_options = pa.csv.ParseOptions(delimiter='|')
lineitem_table = pa.csv.read_csv(input_file='lineitem.tbl', parse_options=csv_options)
lineitem_table

In [ ]:
len(lineitem_table)

In [ ]:
isinstance(lineitem_table, object)

In [ ]:
hex(id(lineitem_table))

In [ ]:
lineitem_table.num_rows

In [ ]:
lineitem_table.num_columns

In [ ]:
id(lineitem_table)

TenDB sample commands

In [ ]:
import tendb as ten

In [ ]:
tcache = ten.new_cache()

In [ ]:
tcache

In [ ]:
tb = ten._tendb.CTenDB()

In [ ]:
tb.show_versions()

In [ ]:
isinstance(lineitem_table, pa.Table)

In [ ]:
tb.add_table(b'lineitem', lineitem_table)

In [ ]:
ten.query5()

In [ ]:
ten.test()

In [ ]:
f = ten.Foo()

In [ ]:
f.bar()